# 🏥 TTSH Hospital-at-Home Nurse Scheduling System
## Complete Beginner-Friendly Guide with OR-Tools

**Author:** Clinical Informatics Team  
**Version:** 1.1 (with Speedoc Vendor Overflow)  
**Last Updated:** February 2026

---

### 📚 What This Notebook Covers

This notebook teaches you how to build an **automated nurse scheduling system** for the Mobile Inpatient Care (MIC) at Home program. We'll go step-by-step from basic concepts to a working prototype.

**Stages:**
1. **Stage 1:** Understanding the Problem & Setup
2. **Stage 2:** Loading and Parsing Excel Data
3. **Stage 3:** Building the Constraint Solver with OR-Tools
4. **Stage 4:** Route Optimization (Solving the Zigzag Problem)
5. **Stage 5:** Visualization and Output
6. **Stage 6:** Streamlit Web Interface

**Key Feature:** When internal nurse capacity is exceeded, overflow visits are automatically routed to **Speedoc Singapore** (external vendor partner).

---

### 🎯 Learning Objectives

By the end of this notebook, you will understand:
- How constraint optimization works (no math PhD required!)
- How to parse messy Excel data into clean structures
- How the Vehicle Routing Problem (VRP) maps to nurse scheduling
- How to handle capacity overflow with vendor partners
- How to deploy your solution on Streamlit Cloud or GitHub Pages

---

## 🚀 STAGE 1: Understanding the Problem & Setup

### 1.1 The Scheduling Challenge

**The Problem:**
- 30-40 patients need visits daily
- 2 nurses available (6 visits each: 3 AM, 3 PM)
- Operating hours: 8:30 AM - 4:30 PM
- 1-hour lunch break between 11:00 AM - 2:00 PM
- Special constraints: IV timings, blood draw deadlines, continuity of care
- **Overflow partner: Speedoc Singapore** for visits exceeding capacity

**Why AI/Algorithms Help:**
- Manual scheduling takes 30-60 minutes daily
- Human planners often create suboptimal routes (zigzagging)
- Constraint violations happen under time pressure

### 1.2 Key Terminology

| Term | Meaning |
|------|--------|
| **OR-Tools** | Google's free Operations Research library for optimization |
| **VRP** | Vehicle Routing Problem - finding optimal routes for multiple vehicles |
| **VRPTW** | VRP with Time Windows - routes must respect time constraints |
| **Constraint** | A rule that must be satisfied (e.g., max 6 visits per nurse) |
| **Objective** | What we want to minimize (e.g., total travel time) |
| **Vendor Overflow** | External partner (Speedoc) handles visits beyond internal capacity |

### 1.3 Installing Required Libraries

Run this cell first to install all dependencies. If you're in Google Colab, these will install in the cloud environment.

In [ ]:
# ============================================================
# INSTALLATION CELL - Run this first!
# ============================================================
# This installs all the libraries we need:
#   - ortools: Google's optimization library (the brain of our scheduler)
#   - pandas: For handling Excel/CSV data (like a programmable spreadsheet)
#   - openpyxl: For reading Excel files
#   - requests: For calling web APIs (like OneMap for geocoding)
#   - folium: For creating interactive maps
#   - streamlit: For building the web interface (Stage 6)
# ============================================================

!pip install ortools pandas openpyxl requests folium streamlit plotly

print("✅ All libraries installed successfully!")

In [ ]:
# ============================================================
# IMPORT LIBRARIES
# ============================================================
# Now we import the libraries we just installed.
# Think of this as "loading tools into your workspace"
# ============================================================

# Data handling
import pandas as pd                    # For Excel/CSV manipulation
import numpy as np                     # For numerical operations
from datetime import datetime, timedelta  # For time calculations
import json                            # For working with JSON data

# Optimization
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Visualization
import folium                          # For interactive maps
from IPython.display import display, HTML  # For displaying in notebook

# Web requests
import requests                        # For API calls

# Utilities
from typing import List, Dict, Tuple, Optional  # Type hints for clarity
from dataclasses import dataclass      # For creating structured data
import warnings
warnings.filterwarnings('ignore')      # Hide unnecessary warnings

print("✅ All libraries imported successfully!")
print(f"📦 OR-Tools version: Check installation complete")

### 1.4 Understanding Data Classes

Before we dive in, let's create **data structures** to organize our information. Think of these as "templates" for storing patient and nurse information in a clean, organized way.

**Why Data Classes?**
- Instead of messy dictionaries, we get clear, named fields
- Easier to understand and debug
- Python checks that we provide required information

In [ ]:
# ============================================================
# DATA STRUCTURES
# ============================================================
# These are like "forms" that define what information we need
# for each patient, nurse, and visit.
# 
# @dataclass is a Python decorator that automatically creates
# a class with the fields we specify. It's like a smart template.
# ============================================================

@dataclass
class Patient:
    """
    Represents a patient who needs home visits.
    """
    id: str
    name: str
    address: str
    postal_code: str
    zone: str = ""
    latitude: float = 0.0
    longitude: float = 0.0
    language: str = "English"


@dataclass
class Visit:
    """
    Represents a single visit that needs to be scheduled.
    A patient may have multiple visits (e.g., AM and PM for 8-hr IV).
    """
    id: str
    patient: Patient
    procedure: str
    session: str  # "AM" or "PM"
    earliest_time: int  # Minutes from midnight
    latest_time: int
    duration_minutes: int = 30
    priority: int = 3  # 1=highest
    requires_continuity: bool = False
    continuity_group: str = ""


@dataclass
class Nurse:
    """
    Represents a nurse available for scheduling.
    """
    id: str
    name: str
    languages: List[str]
    max_visits_am: int = 3
    max_visits_pm: int = 3
    preferred_zones: List[str] = None
    is_vendor: bool = False
    
    def __post_init__(self):
        if self.preferred_zones is None:
            self.preferred_zones = ["North", "South", "East", "West", "Central"]


@dataclass
class ScheduledVisit:
    """
    The result of scheduling - a visit assigned to a nurse at a specific time.
    """
    visit: Visit
    nurse: Nurse
    scheduled_time: int
    travel_time_from_previous: int = 0
    sequence: int = 0


print("✅ Data structures defined!")
print("")
print("📋 We now have templates for:")
print("   - Patient (who needs care)")
print("   - Visit (what needs to be done)")
print("   - Nurse (who provides care, including vendors)")
print("   - ScheduledVisit (the final assignment)")

### 1.5 Configuration Constants

These are the **business rules** encoded as constants.

In [ ]:
# ============================================================
# CONFIGURATION CONSTANTS
# ============================================================

class Config:
    """Central configuration for the scheduling system."""
    
    # === TIME BOUNDARIES ===
    WORK_START = 8 * 60 + 30      # 8:30 AM = 510 minutes
    WORK_END = 16 * 60 + 30       # 4:30 PM = 990 minutes
    LUNCH_WINDOW_START = 11 * 60  # 11:00 AM
    LUNCH_WINDOW_END = 14 * 60    # 2:00 PM
    LUNCH_DURATION = 60
    
    # === CAPACITY CONSTRAINTS ===
    MAX_VISITS_PER_NURSE_AM = 3
    MAX_VISITS_PER_NURSE_PM = 3
    MAX_VISITS_PER_NURSE_TOTAL = 6
    
    # === VISIT DURATIONS (in minutes) ===
    DEFAULT_VISIT_DURATION = 30
    IV_VISIT_DURATION = 45
    BLOOD_DRAW_DURATION = 20
    
    # === BLOOD DRAW CONSTRAINTS ===
    BLOOD_LAB_DEADLINE = 11 * 60  # 11:00 AM
    BLOOD_TRANSIT_TIME = 60       # 1 hour to transport
    BLOOD_DRAW_LATEST = BLOOD_LAB_DEADLINE - BLOOD_TRANSIT_TIME  # 10:00 AM
    
    # === TRAVEL TIME DEFAULTS ===
    DEFAULT_TRAVEL_TIME = 20
    SAME_ZONE_TRAVEL_TIME = 15
    HOSPITAL_RETURN_TIME = 30
    
    # === HOSPITAL LOCATION (TTSH) ===
    HOSPITAL_LAT = 1.3214
    HOSPITAL_LNG = 103.8456
    
    # === VENDOR PARTNER ===
    VENDOR_NAME = "Speedoc Singapore"
    VENDOR_CONTACT = "1800-888-1010"
    
    # === ZONE DEFINITIONS ===
    ZONE_MAPPING = {
        "North": ["50", "51", "52", "53", "54", "55", "56", "57", "72", "73"],
        "South": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"],
        "East": ["38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49"],
        "West": ["60", "61", "62", "63", "64", "65", "66", "67", "68", "69", "70", "71"],
        "Central": ["11", "12", "13", "14", "15", "16", "17", "18", "19", "20", 
                   "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
                   "31", "32", "33", "34", "35", "36", "37"]
    }


def minutes_to_time_string(minutes: int) -> str:
    """Convert minutes from midnight to readable time string."""
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours:02d}:{mins:02d}"


def time_string_to_minutes(time_str: str) -> int:
    """Convert time string to minutes from midnight."""
    time_str = time_str.strip().upper()
    is_pm = "PM" in time_str
    is_am = "AM" in time_str
    time_str = time_str.replace("AM", "").replace("PM", "").strip()
    
    if ":" in time_str:
        parts = time_str.split(":")
        hours = int(parts[0])
        mins = int(parts[1]) if len(parts) > 1 else 0
    else:
        hours = int(time_str)
        mins = 0
    
    if is_pm and hours != 12:
        hours += 12
    if is_am and hours == 12:
        hours = 0
    
    return hours * 60 + mins


print("✅ Configuration loaded!")
print(f"   Work hours: {minutes_to_time_string(Config.WORK_START)} - {minutes_to_time_string(Config.WORK_END)}")
print(f"   Vendor partner: {Config.VENDOR_NAME}")

---

## 📊 STAGE 2: Loading and Parsing Excel Data

### 2.1 Sample Data Creation

In [ ]:
# ============================================================
# SAMPLE DATA CREATION
# ============================================================

def create_sample_data() -> pd.DataFrame:
    """Creates sample patient data matching the TTSH Excel format."""
    
    data = {
        "Name": [
            "Tan AH", "Lim BK", "Wong CL", "Chen DM", "Lee EF",
            "Ng GH", "Goh IJ", "Koh KL", "Teo MN", "Ong PQ"
        ],
        "Location": [
            "Blk 123 Ang Mo Kio Ave 4 #08-123 S(560123)",
            "Blk 456 Toa Payoh Lor 1 #05-456 S(310456)",
            "Blk 789 Hougang Ave 5 #12-789 S(530789)",
            "Blk 234 Bishan St 22 #03-234 S(570234)",
            "Blk 567 Woodlands Dr 14 #07-567 S(730567)",
            "Blk 890 Ang Mo Kio Ave 10 #11-890 S(560890)",
            "Blk 345 Toa Payoh Lor 8 #02-345 S(310345)",
            "Blk 678 Serangoon Ave 2 #09-678 S(550678)",
            "Blk 901 Bishan St 11 #06-901 S(570901)",
            "Blk 432 Ang Mo Kio Ave 1 #04-432 S(560432)"
        ],
        "Home Visit task/time": [
            "IV ABx 8 hrly", "Blood taking", "IV ABx", "Wound dressing",
            "Others (Priority) 10:00", "IV ABx", "Blood taking",
            "IV ABx 8 hrly", "Wound dressing", "Vital signs"
        ],
        "Session 2 task/time": [
            "IV ABx 8 hrly (PM)", "", "", "", "",
            "", "", "IV ABx 8 hrly (PM)", "", ""
        ],
        "Priority": [
            "Normal", "Normal", "Normal", "Normal", "Priority",
            "Normal", "Normal", "Normal", "Normal", "Normal"
        ],
        "Language": [
            "Mandarin", "English", "English", "Mandarin", "Malay",
            "English", "Mandarin", "English", "English", "Mandarin"
        ],
        "Notes": [
            "Prefer morning slot", "Fasting blood test", "", "Diabetic foot ulcer",
            "Needs interpreter", "", "Fasting blood test", "PICC line",
            "Post-surgical", "Elderly, lives alone"
        ]
    }
    
    return pd.DataFrame(data)


sample_df = create_sample_data()
print("✅ Sample data created!")
print(f"📊 Total patients: {len(sample_df)}")
display(sample_df)

### 2.2 Excel Parser

In [ ]:
# ============================================================
# EXCEL PARSER CLASS
# ============================================================

class ExcelParser:
    """Parses TTSH@Home Excel files into structured Patient and Visit objects."""
    
    PROCEDURE_TYPES = {
        "iv abx 8 hrly": {"type": "IV_8HR", "duration": 45, "needs_pair": True},
        "iv abx 12 hrly": {"type": "IV_12HR", "duration": 45, "needs_pair": True},
        "iv abx": {"type": "IV", "duration": 45, "needs_pair": False},
        "blood taking": {"type": "BLOOD", "duration": 20, "needs_pair": False},
        "blood draw": {"type": "BLOOD", "duration": 20, "needs_pair": False},
        "wound dressing": {"type": "WOUND", "duration": 30, "needs_pair": False},
        "vital signs": {"type": "VITALS", "duration": 20, "needs_pair": False},
        "others": {"type": "OTHER", "duration": 30, "needs_pair": False},
    }
    
    def __init__(self):
        self.patients = []
        self.visits = []
        self.warnings = []
    
    def parse_dataframe(self, df: pd.DataFrame) -> Tuple[List[Patient], List[Visit]]:
        """Parse a DataFrame into patients and visits."""
        self.patients = []
        self.visits = []
        self.warnings = []
        
        for idx, row in df.iterrows():
            try:
                patient = self._parse_patient_row(row, idx)
                self.patients.append(patient)
                visits = self._parse_visits_for_patient(patient, row, idx)
                self.visits.extend(visits)
            except Exception as e:
                self.warnings.append(f"Row {idx}: Error parsing - {str(e)}")
        
        return self.patients, self.visits
    
    def _parse_patient_row(self, row: pd.Series, idx: int) -> Patient:
        name = str(row.get("Name", f"Patient_{idx}")).strip()
        address = str(row.get("Location", "")).strip()
        postal_code = self._extract_postal_code(address)
        zone = self._determine_zone(postal_code)
        language = str(row.get("Language", "English")).strip()
        if language.lower() in ["nan", "", "none"]:
            language = "English"
        
        return Patient(id=f"P{idx:03d}", name=name, address=address,
                      postal_code=postal_code, zone=zone, language=language)
    
    def _extract_postal_code(self, address: str) -> str:
        import re
        match = re.search(r'S\((\d{6})\)', address)
        if match:
            return match.group(1)
        match = re.search(r'\b(\d{6})\b', address)
        if match:
            return match.group(1)
        return "000000"
    
    def _determine_zone(self, postal_code: str) -> str:
        prefix = postal_code[:2]
        for zone, prefixes in Config.ZONE_MAPPING.items():
            if prefix in prefixes:
                return zone
        return "Central"
    
    def _parse_visits_for_patient(self, patient: Patient, row: pd.Series, idx: int) -> List[Visit]:
        visits = []
        task1 = str(row.get("Home Visit task/time", "")).strip()
        if task1 and task1.lower() not in ["nan", "", "none"]:
            visit1 = self._create_visit(patient, task1, "AM", idx, 1)
            visits.append(visit1)
            
            proc_info = self._identify_procedure(task1)
            if proc_info.get("needs_pair", False):
                visit2 = self._create_visit(patient, task1, "PM", idx, 2)
                visit2.requires_continuity = True
                visit2.continuity_group = f"CG{idx:03d}"
                visit1.requires_continuity = True
                visit1.continuity_group = f"CG{idx:03d}"
                visits.append(visit2)
        
        return visits
    
    def _create_visit(self, patient: Patient, task: str, session: str,
                      patient_idx: int, visit_num: int) -> Visit:
        proc_info = self._identify_procedure(task)
        earliest, latest = self._calculate_time_window(proc_info, task, session)
        priority = 1 if "priority" in task.lower() else 3
        
        specific_time = self._extract_specific_time(task)
        if specific_time:
            earliest = specific_time
            latest = specific_time + 30
        
        return Visit(id=f"V{patient_idx:03d}_{visit_num}", patient=patient,
                    procedure=proc_info["type"], session=session,
                    earliest_time=earliest, latest_time=latest,
                    duration_minutes=proc_info["duration"], priority=priority)
    
    def _identify_procedure(self, task: str) -> Dict:
        task_lower = task.lower()
        for pattern, info in self.PROCEDURE_TYPES.items():
            if pattern in task_lower:
                return info
        return {"type": "OTHER", "duration": 30, "needs_pair": False}
    
    def _calculate_time_window(self, proc_info: Dict, task: str, session: str) -> Tuple[int, int]:
        proc_type = proc_info["type"]
        if proc_type == "BLOOD":
            return (Config.WORK_START, Config.BLOOD_DRAW_LATEST)
        elif proc_type == "IV_8HR":
            if session == "AM":
                return (Config.WORK_START, 10 * 60)
            else:
                return (16 * 60, Config.WORK_END)
        else:
            if session == "AM":
                return (Config.WORK_START, Config.LUNCH_WINDOW_START)
            else:
                return (Config.LUNCH_WINDOW_END, Config.WORK_END)
    
    def _extract_specific_time(self, task: str) -> Optional[int]:
        import re
        match = re.search(r'(\d{1,2}):(\d{2})', task)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2))
            if hours < 8:
                hours += 12
            return hours * 60 + minutes
        return None


print("✅ ExcelParser class defined!")

In [ ]:
# ============================================================
# TEST THE PARSER
# ============================================================

parser = ExcelParser()
patients, visits = parser.parse_dataframe(sample_df)

print("✅ Parsing complete!")
print(f"📊 Results: {len(patients)} patients, {len(visits)} visits")
print("")
print("📋 VISITS:")
for v in visits:
    time_window = f"{minutes_to_time_string(v.earliest_time)}-{minutes_to_time_string(v.latest_time)}"
    continuity = "🔗" if v.requires_continuity else "  "
    print(f"   {v.id}: {v.patient.name:15} | {v.procedure:10} | {v.session} | {time_window} | {continuity}")

---

## 🧠 STAGE 3: Building the Constraint Solver

### 3.2 Building the Scheduler (with Vendor Overflow)

In [ ]:
# ============================================================
# NURSE SCHEDULER CLASS (with Vendor Overflow)
# ============================================================

class NurseScheduler:
    """
    Main scheduling engine with vendor overflow support.
    When internal capacity is exceeded, overflow goes to Speedoc.
    """
    
    def __init__(self, nurses: List[Nurse], visits: List[Visit],
                 vendor_name: str = "Speedoc Singapore"):
        self.nurses = nurses
        self.visits = visits
        self.scheduled_visits = []
        self.vendor_visits = []
        self.solution = False
        
        self.vendor = Nurse(
            id="V001", name=vendor_name,
            languages=["English", "Mandarin", "Malay", "Tamil"],
            max_visits_am=20, max_visits_pm=20, is_vendor=True
        )
    
    def solve(self, time_limit_seconds: int = 30) -> bool:
        """Solve the scheduling problem with vendor overflow."""
        if len(self.visits) == 0:
            print("⚠️ No visits to schedule!")
            return False
        
        self.scheduled_visits = []
        self.vendor_visits = []
        
        # Sort by priority, then time window tightness
        sorted_visits = sorted(self.visits, key=lambda v: (
            v.priority,
            v.latest_time - v.earliest_time,
            v.earliest_time
        ))
        
        nurse_schedules = {n.id: {"AM": [], "PM": []} for n in self.nurses}
        
        for visit in sorted_visits:
            assigned = False
            session = visit.session
            
            for nurse in self.nurses:
                max_for_session = nurse.max_visits_am if session == "AM" else nurse.max_visits_pm
                current_count = len(nurse_schedules[nurse.id][session])
                
                if current_count >= max_for_session:
                    continue
                
                # Check continuity constraint
                if visit.requires_continuity and visit.continuity_group:
                    related = None
                    for sv in self.scheduled_visits:
                        if (sv.visit.continuity_group == visit.continuity_group and
                            sv.visit.id != visit.id):
                            related = sv
                            break
                    if related and related.nurse.id != nurse.id:
                        continue
                
                # Calculate scheduled time
                if nurse_schedules[nurse.id][session]:
                    last_sv = nurse_schedules[nurse.id][session][-1]
                    last_end = last_sv.scheduled_time + last_sv.visit.duration_minutes
                    travel = (Config.SAME_ZONE_TRAVEL_TIME 
                             if last_sv.visit.patient.zone == visit.patient.zone 
                             else Config.DEFAULT_TRAVEL_TIME)
                    scheduled_time = last_end + travel
                else:
                    travel = Config.HOSPITAL_RETURN_TIME
                    scheduled_time = visit.earliest_time
                
                scheduled_time = max(scheduled_time, visit.earliest_time)
                
                if scheduled_time <= visit.latest_time:
                    sv = ScheduledVisit(
                        visit=visit, nurse=nurse,
                        scheduled_time=scheduled_time,
                        travel_time_from_previous=travel,
                        sequence=len(nurse_schedules[nurse.id][session])
                    )
                    self.scheduled_visits.append(sv)
                    nurse_schedules[nurse.id][session].append(sv)
                    assigned = True
                    break
            
            if not assigned:
                self.vendor_visits.append(visit)
        
        self.solution = True
        return True
    
    def get_schedule_by_nurse(self) -> Dict[str, List[ScheduledVisit]]:
        schedule = {nurse.id: [] for nurse in self.nurses}
        schedule[self.vendor.id] = []
        
        for sv in self.scheduled_visits:
            schedule[sv.nurse.id].append(sv)
        
        for i, v in enumerate(self.vendor_visits):
            sv = ScheduledVisit(
                visit=v, nurse=self.vendor,
                scheduled_time=v.earliest_time,
                travel_time_from_previous=0, sequence=i
            )
            schedule[self.vendor.id].append(sv)
        
        for nurse_id in schedule:
            schedule[nurse_id].sort(key=lambda x: x.scheduled_time)
        
        return schedule
    
    def calculate_metrics(self) -> Dict:
        return {
            'total_visits': len(self.scheduled_visits) + len(self.vendor_visits),
            'internal_visits': len(self.scheduled_visits),
            'vendor_visits': len(self.vendor_visits),
            'total_travel_time': sum(sv.travel_time_from_previous for sv in self.scheduled_visits),
            'visits_per_nurse': {
                n.name: len([sv for sv in self.scheduled_visits if sv.nurse.id == n.id])
                for n in self.nurses
            },
            'unassigned_visits': 0
        }


print("✅ NurseScheduler class defined!")
print("   - Prioritizes internal nurses")
print("   - Routes overflow to Speedoc vendor")

### 3.3 Running the Scheduler

In [ ]:
# ============================================================
# CREATE NURSES AND RUN SCHEDULER (WITH VENDOR OVERFLOW)
# ============================================================

nurses = [
    Nurse(id="N001", name="Nurse Alice", languages=["English", "Mandarin"],
          preferred_zones=["North", "Central"]),
    Nurse(id="N002", name="Nurse Betty", languages=["English", "Malay"],
          preferred_zones=["Central", "East"])
]

print("👩‍⚕️ Available Staff:")
for nurse in nurses:
    print(f"   {nurse.id}: {nurse.name} - Languages: {nurse.languages}")
print(f"   V001: {Config.VENDOR_NAME} (External Vendor - Overflow)")

print("")
print("📊 Capacity Analysis:")
total_visits = len(visits)
am_visits = len([v for v in visits if v.session == "AM"])
pm_visits = len([v for v in visits if v.session == "PM"])
nurse_capacity = len(nurses) * Config.MAX_VISITS_PER_NURSE_TOTAL

print(f"   Total visits needed: {total_visits}")
print(f"   AM visits: {am_visits}, PM visits: {pm_visits}")
print(f"   Internal capacity: {nurse_capacity} ({len(nurses)} nurses × {Config.MAX_VISITS_PER_NURSE_TOTAL})")

if total_visits > nurse_capacity:
    print(f"   ⚠️ OVERFLOW EXPECTED: ~{total_visits - nurse_capacity} visits may go to vendor")

print("")
print("🔄 Running scheduler...")
print("="*60)

scheduler = NurseScheduler(nurses=nurses, visits=visits, vendor_name=Config.VENDOR_NAME)
success = scheduler.solve(time_limit_seconds=30)

print("")
print("✅ SCHEDULING COMPLETE!")
print("="*60)

metrics = scheduler.calculate_metrics()
print(f"")
print(f"📊 METRICS:")
print(f"   Total visits: {metrics['total_visits']}")
print(f"   ├─ Internal nurses: {metrics['internal_visits']}")
print(f"   └─ {Config.VENDOR_NAME}: {metrics['vendor_visits']}")
print(f"   Total travel time (internal): {metrics['total_travel_time']} minutes")
print(f"")
print(f"   Visits per nurse:")
for nurse_name, count in metrics['visits_per_nurse'].items():
    print(f"      {nurse_name}: {count} visits")
if scheduler.vendor_visits:
    print(f"      {Config.VENDOR_NAME}: {len(scheduler.vendor_visits)} visits")

print("")
print("="*60)
print("📋 DETAILED SCHEDULE:")
print("="*60)

schedule = scheduler.get_schedule_by_nurse()

for nurse in nurses:
    nurse_visits = schedule[nurse.id]
    print(f"")
    print(f"👩‍⚕️ {nurse.name}:")
    print(f"   {'-'*50}")
    
    if not nurse_visits:
        print(f"   No visits assigned")
    else:
        for sv in nurse_visits:
            time_str = minutes_to_time_string(sv.scheduled_time)
            end_time_str = minutes_to_time_string(sv.scheduled_time + sv.visit.duration_minutes)
            print(f"   {sv.sequence+1}. [{time_str}-{end_time_str}] {sv.visit.patient.name}")
            print(f"      📍 {sv.visit.patient.zone} | {sv.visit.procedure} | Travel: {sv.travel_time_from_previous} min")

if scheduler.vendor_visits:
    print("")
    print("="*60)
    print(f"📞 {Config.VENDOR_NAME.upper()} HANDOFF REQUIRED:")
    print("="*60)
    print(f"Contact: {Config.VENDOR_CONTACT}")
    print("")
    print("Please arrange the following visits with Speedoc:")
    print("-"*60)
    
    for i, v in enumerate(scheduler.vendor_visits):
        print(f"")
        print(f"   {i+1}. {v.patient.name}")
        print(f"      📍 {v.patient.address}")
        print(f"      🏥 Procedure: {v.procedure}")
        print(f"      ⏰ Time window: {minutes_to_time_string(v.earliest_time)}-{minutes_to_time_string(v.latest_time)}")
        print(f"      🗣️ Language: {v.patient.language}")
else:
    print("")
    print("✅ All visits handled by internal nurses - no vendor needed!")

---

## 🗺️ STAGE 4: Route Visualization

In [ ]:
# ============================================================
# GEOCODING WITH ONEMAP API
# ============================================================

class OneMapGeocoder:
    """Geocoder using Singapore's OneMap API."""
    
    BASE_URL = "https://www.onemap.gov.sg/api/common/elastic/search"
    
    def __init__(self):
        self.cache = {}
    
    def geocode(self, address: str) -> Tuple[float, float]:
        if address in self.cache:
            return self.cache[address]
        
        import re
        postal_match = re.search(r'\d{6}', address)
        search_term = postal_match.group() if postal_match else address
        
        try:
            response = requests.get(
                self.BASE_URL,
                params={'searchVal': search_term, 'returnGeom': 'Y', 'getAddrDetails': 'Y'},
                timeout=10
            )
            
            if response.status_code == 200:
                data = response.json()
                if data.get('found', 0) > 0:
                    result = data['results'][0]
                    lat = float(result['LATITUDE'])
                    lng = float(result['LONGITUDE'])
                    self.cache[address] = (lat, lng)
                    return (lat, lng)
        except Exception as e:
            print(f"⚠️ Geocoding error: {e}")
        
        return (Config.HOSPITAL_LAT, Config.HOSPITAL_LNG)
    
    def geocode_patients(self, patients: List[Patient]) -> List[Patient]:
        print("🗺️ Geocoding patient addresses...")
        for i, patient in enumerate(patients):
            lat, lng = self.geocode(patient.address)
            patient.latitude = lat
            patient.longitude = lng
            print(f"   {i+1}/{len(patients)}: {patient.name} -> ({lat:.4f}, {lng:.4f})")
        print("✅ Geocoding complete!")
        return patients


geocoder = OneMapGeocoder()
patients = geocoder.geocode_patients(patients)

In [ ]:
# ============================================================
# ROUTE VISUALIZATION
# ============================================================

def create_route_map(scheduler: NurseScheduler) -> folium.Map:
    """Create an interactive map showing nurse routes."""
    m = folium.Map(
        location=[Config.HOSPITAL_LAT, Config.HOSPITAL_LNG],
        zoom_start=12, tiles='CartoDB positron'
    )
    
    folium.Marker(
        [Config.HOSPITAL_LAT, Config.HOSPITAL_LNG],
        popup='🏥 TTSH (Start/End)',
        icon=folium.Icon(color='red', icon='plus', prefix='fa')
    ).add_to(m)
    
    nurse_colors = ['blue', 'green', 'purple', 'orange', 'darkred']
    schedule = scheduler.get_schedule_by_nurse()
    
    for nurse_idx, nurse in enumerate(scheduler.nurses):
        color = nurse_colors[nurse_idx % len(nurse_colors)]
        nurse_visits = schedule[nurse.id]
        
        if not nurse_visits:
            continue
        
        route_coords = [[Config.HOSPITAL_LAT, Config.HOSPITAL_LNG]]
        
        for sv in nurse_visits:
            lat, lng = sv.visit.patient.latitude, sv.visit.patient.longitude
            if lat != 0 and lng != 0:
                route_coords.append([lat, lng])
                time_str = minutes_to_time_string(sv.scheduled_time)
                popup_text = f"<b>{sv.visit.patient.name}</b><br>Time: {time_str}<br>Nurse: {nurse.name}"
                folium.Marker(
                    [lat, lng], popup=folium.Popup(popup_text, max_width=200),
                    icon=folium.Icon(color=color, icon='user', prefix='fa')
                ).add_to(m)
        
        route_coords.append([Config.HOSPITAL_LAT, Config.HOSPITAL_LNG])
        folium.PolyLine(route_coords, weight=3, color=color, opacity=0.7).add_to(m)
    
    # Vendor visits
    if scheduler.vendor_visits:
        for v in scheduler.vendor_visits:
            lat, lng = v.patient.latitude, v.patient.longitude
            if lat != 0 and lng != 0:
                popup_text = f"<b>{v.patient.name}</b><br><span style='color:orange'>⚠️ VENDOR</span>"
                folium.Marker(
                    [lat, lng], popup=folium.Popup(popup_text, max_width=200),
                    icon=folium.Icon(color='gray', icon='ambulance', prefix='fa')
                ).add_to(m)
    
    return m


if scheduler.solution:
    route_map = create_route_map(scheduler)
    print("✅ Map created!")
    display(route_map)

---

## 📄 STAGE 5: Export to Excel

In [ ]:
# ============================================================
# EXCEL EXPORT
# ============================================================

def export_schedule_to_excel(scheduler: NurseScheduler, filename: str = "schedule_output.xlsx"):
    """Export the schedule to Excel file."""
    schedule = scheduler.get_schedule_by_nurse()
    
    internal_rows = []
    for nurse in scheduler.nurses:
        for sv in schedule[nurse.id]:
            internal_rows.append({
                'Nurse': nurse.name,
                'Sequence': sv.sequence + 1,
                'Scheduled Time': minutes_to_time_string(sv.scheduled_time),
                'End Time': minutes_to_time_string(sv.scheduled_time + sv.visit.duration_minutes),
                'Patient Name': sv.visit.patient.name,
                'Location': sv.visit.patient.address,
                'Zone': sv.visit.patient.zone,
                'Procedure': sv.visit.procedure,
                'Session': sv.visit.session,
                'Travel Time (min)': sv.travel_time_from_previous
            })
    
    vendor_rows = [{
        'Patient Name': v.patient.name,
        'Location': v.patient.address,
        'Procedure': v.procedure,
        'Time Window': f"{minutes_to_time_string(v.earliest_time)}-{minutes_to_time_string(v.latest_time)}",
        'Language': v.patient.language
    } for v in scheduler.vendor_visits]
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        pd.DataFrame(internal_rows).to_excel(writer, sheet_name='Internal Schedule', index=False)
        if vendor_rows:
            pd.DataFrame(vendor_rows).to_excel(writer, sheet_name='Speedoc Handoff', index=False)
    
    print(f"✅ Schedule exported to '{filename}'")
    if vendor_rows:
        print(f"   ⚠️ Includes {len(vendor_rows)} visits for Speedoc handoff")
    return pd.DataFrame(internal_rows)


if scheduler.solution:
    export_df = export_schedule_to_excel(scheduler, "schedule_output.xlsx")
    display(export_df)

---

## 🎉 COMPLETE WORKING EXAMPLE

In [ ]:
# ============================================================
# COMPLETE END-TO-END EXAMPLE
# ============================================================

def run_complete_scheduler(excel_path: str = None):
    """Run the complete scheduling pipeline."""
    print("="*60)
    print("🏥 TTSH@Home Nurse Scheduling System")
    print(f"   Vendor Partner: {Config.VENDOR_NAME}")
    print("="*60)
    
    parser = ExcelParser()
    if excel_path:
        patients, visits = parser.parse_excel(excel_path)
    else:
        sample_df = create_sample_data()
        patients, visits = parser.parse_dataframe(sample_df)
    
    print(f"✓ Loaded {len(patients)} patients, {len(visits)} visits")
    
    nurses = [
        Nurse(id="N001", name="Nurse Alice", languages=["English", "Mandarin"]),
        Nurse(id="N002", name="Nurse Betty", languages=["English", "Malay"])
    ]
    
    scheduler = NurseScheduler(nurses=nurses, visits=visits, vendor_name=Config.VENDOR_NAME)
    scheduler.solve()
    
    metrics = scheduler.calculate_metrics()
    print(f"\n📊 Results:")
    print(f"   Internal: {metrics['internal_visits']} | Vendor: {metrics['vendor_visits']}")
    
    schedule = scheduler.get_schedule_by_nurse()
    for nurse in nurses:
        nurse_visits = schedule[nurse.id]
        print(f"\n{nurse.name}: {len(nurse_visits)} visits")
        for sv in nurse_visits:
            print(f"  {sv.sequence+1}. [{minutes_to_time_string(sv.scheduled_time)}] {sv.visit.patient.name}")
    
    if scheduler.vendor_visits:
        print(f"\n⚠️ {Config.VENDOR_NAME}: {len(scheduler.vendor_visits)} visits")
        for v in scheduler.vendor_visits:
            print(f"  • {v.patient.name} - {v.procedure}")
    
    print("\n✅ Scheduling complete!")
    return scheduler


result = run_complete_scheduler()

---

## 📚 Next Steps

**Deployment options:**
- `streamlit_app.py` - Web interface for Streamlit Cloud
- `docs/index.html` - Static version for GitHub Pages

**To deploy:**
1. Push all files to GitHub
2. Connect to Streamlit Cloud or enable GitHub Pages
3. Share the link with your team!

---

*Created for TTSH Mobile Inpatient Care at Home*  
*Vendor Partner: Speedoc Singapore*